# Duan and Zhang (2006)

H2O-CO2 model of Duan, Z, Zhang, Z (2006) Equation of state of the H2O, CO2, and H2O-CO2 systems up to 10 GPa and 2573.15 K: Molecular dynamics simulations with ab initio potential surface. Geochimica et Cosmochimica Acta, 70, 2311-2324  

Read in ENKI phases module

In [ ]:
from thermoengine import phases
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt
%matplotlib inline

Create an instance of Duan and Zhang

In [ ]:
DuanAndZhang = phases.SolutionPhase('FluidDuan', '', calib=False)

In [ ]:
DuanCO2 = phases.PurePhase('DuanCO2', '', calib=False)

## Some calculations for evaulating Duan and Zhang against experiments

Reaction: C,graphite + O<sub>2</sub>,g -> CO<sub>2</sub>,fluid at 1 GPa and 800°C  

The following apparent free energy values are from SUPCRT (CHONSZ) and follow the Helgeson reference state condition

In [ ]:
DuanAndZhang.MINVAL = np.finfo(float).eps*10.0
t = 1073.15
p = 10000.
G_O2 = -42463.88528*4.184
G_graphite = -1913.037*4.184
G_CO2_Duan = DuanAndZhang.chem_potential(1073.15, 10000, mol=np.array([0.0, 1.0]))[0][1] + 62876.8535
deltaG = G_CO2_Duan - G_O2 - G_graphite
G_O2, G_graphite, G_CO2_Duan, deltaG, -deltaG/np.log(10.0)/8.3143/t

In [ ]:
DuanAndZhang.activity(1073.15, 10000, mol=np.array([0.0, 1.0]))

In [ ]:
DuanAndZhang.chem_potential(1073.15, 10000, mol=np.array([0.0, 1.0]))

In [ ]:
t = 1073.15
p = 10000.
G_O2 = -42463.88528*4.184
G_graphite = -1913.037*4.184
G_CO2_Duan = DuanAndZhang.gibbs_energy(1073.15, 10000, mol=np.array([0.0, 1.0])) + 62876.8535
deltaG = G_CO2_Duan - G_O2 - G_graphite
G_O2, G_graphite, G_CO2_Duan, deltaG, -deltaG/np.log(10.0)/8.3143/t

In [ ]:
t = 1073.15
p = 10000.
G_O2 = -42463.88528*4.184
G_graphite = -1913.037*4.184
G_CO2_Duan = DuanCO2.gibbs_energy(t,p) + 62876.8535
deltaG = G_CO2_Duan - G_O2 - G_graphite
G_O2, G_graphite, G_CO2_Duan, deltaG, -deltaG/np.log(10.0)/8.3143/t

At the QFM oxygen buffer at this T,P, the experimentally determined composition of the fluid should yield an activity of CO<sub>2</sub> of 0.67.  This calibration of QFM is from Frost.

In [ ]:
log10fO2 = -24441.9/t + 0.110*(p-1.0)/t +  8.290
log10fO2, log10fO2*1.055, log10fO2*1.055 - log10fO2

Plot the free energy change of the above reaction vrs X CO<sub>2</sub> in the fluid.  Note that the chemical potential of CO<sub>2</sub> is corrected by 62876.8535 J/mol in order to reconcile the Berman (MELTS) reference state value to the Helgeson convention (62876.8535 is the entrop of C + O2 in the reference state multiplied by 298.15 K)

In [ ]:
log10fO2 = -13.70
deltaQFM_FM19  = -0.69 + 0.24 # Frost = first number
deltaQFM_COH69 = -0.78 + 0.24 # Frost = first number
print ('COH69 log10 fO2 = {0:5.2f}'.format(log10fO2+deltaQFM_COH69))
print ('FM19  log10 fO2 = {0:5.2f}'.format(log10fO2+deltaQFM_FM19))
fig = plt.figure(figsize=(15,10))
plt.subplot(1,1,1)
plt.title('Duan and Zhang (2006) P = '+str(p)+' bars'+' T = '+str(t-273.15)+' °C')
x = []
y_FM19  = []
y_COH69 = []
for ix in range(1,100):
    X = ix*0.01
    res = DuanAndZhang.chem_potential(1073.15, 10000, mol=np.array([1.-X, X]))[0][1] + 62876.8535
    res -=  G_O2 + 8.3143*t*np.log(10.0)*log10fO2
    res -=  G_graphite
    y_COH69.append(res - 8.3143*t*np.log(10.0)*deltaQFM_COH69)
    y_FM19.append( res - 8.3143*t*np.log(10.0)*deltaQFM_FM19)
    x.append(X)
plt.plot(np.array(x), np.array(y_COH69), 'r-', label='COH69 delta QFM = '+str(deltaQFM_COH69))
plt.plot(np.array(x), np.array(y_FM19), 'g-', label='FM19 delta QFM = '+str(deltaQFM_FM19))
plt.plot(np.array([0,1]), np.array([0,0]), 'k--')
plt.plot(np.array([0.67, 0.67]), np.array([-1000, 1000]), 'k-')
plt.plot(np.array([0.86, 0.86]), np.array([-1000, 1000]), 'k-')
plt.xlabel('X CO2')
plt.ylabel('deltaG (J) of C,graphite + O2,g = CO2,fld')
plt.xlim([0,1])
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig("graphite-duan.pdf", bbox_inches='tight')

Trial and error estimation of where the equilibrium concentration

In [ ]:
X = 0.67
res = DuanAndZhang.chem_potential(1073.15, 10000, mol=np.array([1.-X, X]))[0][1] + 62876.8535
res -=  G_O2 + 8.3143*t*np.log(10.0)*(log10fO2 + deltaQFM_COH69)
res -=  G_graphite
res

Experimentally, the value of the activity should be 0.67, which is consistent with a Duan and Zhang fluid molefraction of:

In [ ]:
X = 0.67
DuanAndZhang.activity(t, p, mol=np.array([1.0-X, X]))

In [ ]:
X = 0.86
DuanAndZhang.activity(t, p, mol=np.array([1.0-X, X]))

## Reference state properties and the Helgeson convention

In [ ]:
print ('G  298.15 K,1 bar)', -94254*4.184, 'J/mol')
print ('H  298.15 K,1 bar)', -94051*4.184, 'J/mol')
print ('S  298.15 K,1 bar)', 51.085*4.184, 'J/K-mol')
print ('Cp 298.15 K,1 bar)', 13.51*4.184,  'J/K-mol')

In [ ]:
print ('D&Z Gibbs energy: ', DuanAndZhang.gibbs_energy(298.15, 1.0, mol=np.array([0.0, 1.0])))
print ('D&Z Enthalpy:     ', DuanAndZhang.enthalpy(298.15, 1.0, mol=np.array([0.0, 1.0])))
print ('D&Z Entropy:      ', DuanAndZhang.entropy(298.15, 1.0, mol=np.array([0.0, 1.0])))
print ('D&Z Heat Capacity:', DuanAndZhang.heat_capacity(298.15, 1.0, mol=np.array([0.0, 1.0])))
S_C_ref = 5.74
S_O2_ref = 205.15
print ('Helgeson S corr:  ', 298.15*(S_C_ref+S_O2_ref)) 
print ('D&Z G Helg corr:  ', DuanAndZhang.gibbs_energy(298.15, 1.0, mol=np.array([0.0, 1.0]))+298.15*(S_C_ref+S_O2_ref))

## Density estimates

Set T and P ...

In [ ]:
t = 1000 # K
p = 1000 # bars

Set composition in moles of endmembers ...

In [ ]:
xH2O = 0.2
xCO2 = 0.8

Compute volume in units of J/bar, note that 1 J/bar = 10 cc

In [ ]:
volume = DuanAndZhang.volume(1000, 1000, mol=[xH2O, xCO2])
print ('V =', volume, 'J/bar')

Compute density in g/cc ...

In [ ]:
mwH2O = DuanAndZhang.props['molwt'][0]
mwCO2 = DuanAndZhang.props['molwt'][1]
grams = xH2O*mwH2O + xCO2*mwCO2
density = grams/(volume*10)
print ('density = ', density, 'gm/cc')

## Make some plots of fluid properties over a T-P grid

In [ ]:
for xy in [4]: # range(0,11)
    xCO2 = xy/10.0
    xH2O = 1.0-xCO2
    fig = plt.figure(figsize=(15,10))
    plt.subplot(1,1,1)
    plt.title('Duan and Zhang (2006) X CO2 = '+str(xCO2))
    for p in range(5000,21000,1000):
        x = []
        y = []
        for tc in range(500,801,1):
            t = tc + 273.15
            x.append(tc)
            #y.append(DuanAndZhang.entropy(t, p, mol=[xH2O, xCO2]))
            y.append(DuanAndZhang.volume(t, p, mol=[xH2O, xCO2]))
            #y.append(DuanAndZhang.chem_potential(t, p, mol=[xH2O, xCO2])[0,1])
            #y.append(DuanAndZhang.activity(t, p, mol=[xH2O, xCO2])[0,1])
        plt.plot(np.array(x), np.array(y), '-', label='P = '+str(p))
    plt.xlabel('T °C')
    plt.ylabel('V (J/bar)')
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()
    fig.savefig("DZ-VvrsT.pdf", bbox_inches='tight')

### Make some plots on P-V grid

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.subplot(1,1,1)
xCO2 = 0.4
xH2O = 1.0-xCO2
plt.title('Duan and Zhang (2006) X CO2 = '+str(xCO2))
for tc in range(500,810,10):
    t = tc + 273.15
    x = []
    y = []
    for p in range(5000,20100,100):
        y.append(p/10.0)
        x.append(DuanAndZhang.volume(t, p, mol=[xH2O, xCO2]))
    plt.plot(np.array(x), np.array(y), '-', label='T (°C) = '+str(tc))
plt.xlabel('V (J/bar)')
plt.ylabel('P (MPa)')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()
fig.savefig("DZ-VvrsP.pdf", bbox_inches='tight')

In [ ]:
min_P = 5000
max_P = 20000
inc_P = 25
min_T = 500
max_T = 800
inc_T = 5
for xy in [4]:
    xCO2 = xy/10.0
    xH2O = 1.0-xCO2
    fig = plt.figure(figsize=(15,10))
    plt.subplot(1,1,1)
    plt.title('Duan and Zhang (2006), Volume contours in J/bar, X CO2 = '+str(xCO2))
    x = [tc for tc in range(min_T, max_T+inc_T, inc_T)]
    y = [p/10.0 for p in range(min_P,max_P+inc_P,inc_P)]
    z = []
    for p in range(min_P,max_P+inc_P,inc_P):
        zz = []
        for tc in range(min_T, max_T+inc_T, inc_T):
            t = tc + 273.15
            zz.append(DuanAndZhang.volume(t, p, mol=[xH2O, xCO2]))
        z.append(np.array(zz))
    cs = plt.contour(np.array(x), np.array(y), np.array(z),levels=20)
    plt.clabel(cs, cs.levels, inline=True, fmt='%3.1f', fontsize=10)
    plt.xlabel('T (°C)')
    plt.ylabel('P (MPa)')
    plt.tight_layout()
    plt.show()
    fig.savefig("DZ-TvrsP-contour-V.pdf", bbox_inches='tight')